#### <font color="darkblue"> 1. This script builds data to train custom classification model in Comprehend. <br> 2. The training files are picked in such a way that they representat the topics accurately. <br> 3. The messages with highest confidence (probability) values and more number of keywords (topic-terms) are selected. <br> 4. The confidence and terms are obtained from previous unsupervised model run. <br> 5. Also, the messages which were origninated from CMS RDS Center are not considered. <br> 6. Prerequisites: <br> &nbsp; &nbsp; a) Run /root/comp_predictions/training_data.sh first (This script collects messages with highest confidence scores).

In [ ]:
!python --version

In [ ]:
!pip install psycopg2-binary

In [ ]:
# import os
# import glob
# from pathlib import Path
import boto3
import csv
import pandas as pd
from io import StringIO
import psycopg2

#### <font color="blue"> Functions to read and write to PostgreSQL database tables

In [ ]:
#------------------------------ Function to print errors from PostgreSQL ------------------------------#

def print_psycopg2_exception(err):
    err_type, err_obj, traceback = sys.exc_info()
    line_num = traceback.tb_lineno
    print ("\npsycopg2 ERROR:", err, "on line number:", line_num)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)
    print ("\nextensions.Diagnostics:", err.diag)
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    input("Press Enter to continue...")

In [ ]:
#-------------- Read PostgreSQL and get processed email keys into a list first -------------------------#

def read_postgres(email_key):
    records_in_pgres = []
    try:    
        cursor.execute("SELECT sender FROM ml_extract_track_v2 WHERE email_key = '" + email_key + "'; ")
        row = cursor.fetchone()
        # print('sender:', row[0])

    except Exception as err:
        print_psycopg2_exception(err)

    return row[0]

#### <font color="blue"> Mention s3 bukets and folders to read and write to here

In [ ]:
dl_bucket = "pro-ads-datalake"
s3_file_path = "ml-training-files/"
s3_read_folder = "ml-cleaned-emails/"
s3_write_folder = "ml-training-data/"

cdo_path = "/root/ml_orgtxt_emails/"
cdc_path = "/root/ml_cleaned_emails/"

resource = boto3.resource('s3')
s3 = boto3.client('s3')

bucket = resource.Bucket(dl_bucket)

####  <font color="blue"> 1. Define top 7 keywords (topic-terms) to search from the messages shortlisted in training_data.csv <br> 2. Define s3 folder names to write the training files to

In [ ]:
topic_matches = [ ( "request", "support", "ref", "program", "inquiry", "continue", "response", "resolve" ),
                 ( "account", "password", "login", "log", "lock", "reset", "website", "mfa" ), 
                 ( "account", "manager", "plan", "user", "sponsor", "authorize", "representative", "role" ), 
                 ( "application", "plan", "submit", "sponsor", "id", "deadline", "year", "annual" ), 
                 ( "reconciliation", "step", "complete", "retiree", "cover", "process", "list", "finalize" ), 
                 ( "information", "contact", "sponsor", "additional", "overpayment", "email", "need", "action", "require", "py"),
                 ( "request", "payment", "support", "ref", "setup", "interim", "notification", "hold" ),
                 ( "cost", "report", "submit", "adjustment", "option", "vendor", "reporter", "benefit" ) ]

Old_train_folder = [ "topic1-account-login-password-reset", "topic2-account-manager-auth-rep-reassignment", 
                 "topic3-appl-plan-submission-annual-deadline", "topic4-reconciliation-steps-processing", 
                 "topic5-information-contact-overpayment-inquiries", "topic6-cost-report-vendor-benefit-adjustment", 
                 "topic7-retiree-file-upload-notification", "topic8-support-request-payment-program-resolution" ]

train_folder = [ "topic1-support-ref-program-response-request", "topic2-account-login-password-reset",  
                 "topic3-acct-mgr-auth-rep-reassignments", "topic4-appl-plan-sub-deadline-approval", 
                 "topic5-recon-steps-retiree-process-finalization", "topic6-info-contact-addl-overpayment-req", 
                 "topic7-payment-request-support-setup-notification", "topic8-cost-report-vendor-option-benefit-adj" ]


####  <font color="blue"> Connect to PostgreSQL database

In [ ]:
#------------------------------- Connect to PostgreSQL and read records ---------------------------------#

# cursor.close()
# conn.close()
conn = psycopg2.connect(database="YOUR_DATABASE", user='<YOUR_USER>', password='<YOUR_PASSWORD>', 
                        host='<DATABASE_HOST>', port= '5432')
cursor = conn.cursor()

####  <font color="blue"> Read the filenames from training_data.csv and determine which messages can be considered for training

In [ ]:
cnt = 1
topic_no = 0
topic_words = topic_matches[topic_no]

df = pd.read_csv("/root/comp_predictions/training_data.csv")
# print(df.head(20))
df.sort_values(["topic", "proportion"], inplace=True, ascending=[True, False])
# print(df.head(20))

for i, row in df.iterrows():
    filename = row['docname']
    topicNum = row['topic']
    # print (i, filename, topicNum)
    
    if topic_no != topicNum: 
        continue
    else:
        if cnt < 61: 
            email_key = filename[0:-6]
            sender = read_postgres(email_key)
            sender = sender.lower()
            # print(email_key, " sender:", sender)
            
            if sender.find("cms.hhs.gov") != -1 or sender.find("mailer-daemon") != -1:
                continue 
            
            with open(cdc_path + filename, 'r') as cdcfile:
                fileContent = cdcfile.read()
            
            if len(fileContent) < 100:
                continue
                
            score = 0
            for word in topic_words:
                if fileContent.find(word) != -1:
                    score += 1
            
            # print('topic #:', topic_no, '  score now:', score)
                
            if score >= 4:
                print('file:', email_key, 'sender:', sender)
                # x = input("Hit enter to continue..")
                read_file = cdo_path + filename
                write_path = s3_file_path + train_folder[topic_no]
                read_s3_file = s3_read_folder + filename
                write_s3_file = s3_write_folder + train_folder[topic_no] + "/" + filename
                print(cnt, ") ", filename, dl_bucket, '%s %s %s' % (write_path, read_s3_file, write_s3_file))
                s3.upload_file(read_file, dl_bucket, '%s/%s' % (write_path, filename))
                # input("Enter to continue..")
                copy_source = {'Bucket': dl_bucket, 'Key': read_s3_file}
                resource.meta.client.copy(copy_source, dl_bucket, write_s3_file)
                   
                cnt += 1
                score = 0
            # else:
                # print('score is', score, ' skipping...', score)  
        else:
            # reset count and move to the next topic
            cnt = 1
            topic_no += 1
            if topic_no == 8: break
            topic_words = topic_matches[topic_no]

####  <font color="blue"> This code is a variation of previous code segment, which <br> &nbsp; &nbsp; a) runs for a particular topic number and <br> &nbsp; &nbsp; b) doesn't advance to the next topic automatically when done. 

In [ ]:
# For a specific topic and score lower than 6.. 

cnt = 1
topic_no = 5
topic_words = topic_matches[topic_no]

df = pd.read_csv("/root/comp_predictions/training_data.csv")
# print(df.head(20))
df.sort_values(["topic", "proportion"], inplace=True, ascending=[True, False])
# print(df.head(20))

for i, row in df.iterrows():
    filename = row['docname']
    topicNum = row['topic']
    # print (i, filename, topicNum)
    
    if topic_no != topicNum: 
        continue
    else:
        if cnt < 32: 
            email_key = filename[0:-6]
            sender = read_postgres(email_key)
            sender = sender.lower()
            # print(email_key, " sender:", sender)
            
            if sender.find("cms.hhs.gov") != -1 or sender.find("mailer-daemon") != -1:
                continue 
            
            with open(cdc_path + filename, 'r') as cdcfile:
                fileContent = cdcfile.read()
            
            if len(fileContent) < 100:
                continue
                
            score = 0
            for word in topic_words:
                if fileContent.find(word) != -1:
                    score += 1
            
            # print('topic #:', topic_no, '  score now:', score)
                
            if score >= 3:
                print('file:', email_key, 'sender:', sender)
                # x = input("Hit enter to continue..")
                read_file = cdo_path + filename
                write_path = s3_file_path + train_folder[topic_no]
                read_s3_file = s3_read_folder + filename
                write_s3_file = s3_write_folder + train_folder[topic_no] + "/" + filename
                print(cnt, ") ", filename, dl_bucket, '%s %s %s' % (write_path, read_s3_file, write_s3_file))
                s3.upload_file(read_file, dl_bucket, '%s/%s' % (write_path, filename))
                # input("Enter to continue..")
                copy_source = {'Bucket': dl_bucket, 'Key': read_s3_file}
                resource.meta.client.copy(copy_source, dl_bucket, write_s3_file)
                   
                cnt += 1
                score = 0

#### <font color="blue"> Define output file locations (temporary and final output) for the csv file 

In [ ]:
# csv_file = open("/tmp/train_model_topic1.csv", "w", newline='')
# writer = csv.writer(csv_file, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
# outFile = "ml-training-data-csv/train_model_topic1.csv"

csv_file = open("/tmp/train_model.csv", "w", newline='')
writer = csv.writer(csv_file, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
outFile = "ml-training-data-csv/train_model.csv"

#### <font color="blue"> Build CSV file from the training data written to s3 folder <i>(ml-training-data)</i> for training the custom classification model 

In [ ]:
cnt = 1
for obj in bucket.objects.filter(Prefix='ml-training-data/'):
    key = obj.key
    if (obj.size > 0):
        labelStPos = key.find('/') + 1
        labelEndPos = key.find('/', labelStPos) 
        label = key[labelStPos:labelEndPos]
        # if not label.startswith("topic1"): continue
        fileName = key[labelEndPos+1:] 
        fileContent = obj.get()['Body'].read().decode('utf-8')
        print(str(cnt) + ") ", 'file:', key, '  label:', label)
        fileContent = fileName + " : " + fileContent
        writer.writerow([label, fileContent])
        # input("Enter to continue")
        label = ""
        fileContent = ""
        cnt = cnt + 1
        csv_file.flush()
    # if cnt > 100: break
    
csv_file.close()
data = pd.read_csv("/tmp/train_model.csv", header = None)
shuffled_df = data.sample(frac=1)

csv_buffer = StringIO()
# shuffled_df.to_csv(csv_buffer, index=False, header=False)
shuffled_df.to_csv(csv_buffer, index=False, header=None)
resource.Object(dl_bucket, outFile).put(Body=csv_buffer.getvalue())

####  <font color="blue"> This code is a variation of previous code segment, to run standalone, to read and shuffle the train_model.csv file once generated.

In [ ]:
data = pd.read_csv("/tmp/train_model.csv", header = None)
shuffled_df = data.sample(frac=1)
len(shuffled_df.index)
# shuffled_df.shape[0]
# len(data.index)
csv_buffer = StringIO()
# shuffled_df.to_csv(csv_buffer, index=False, header=False)
shuffled_df.to_csv(csv_buffer, index=False, header=None)
resource.Object(dl_bucket, outFile).put(Body=csv_buffer.getvalue())

####  <font color="blue"> Create custom classifier model using the training data (csv file)<br> Note: Make sure to change the name of the model in <i>DocumentClassifierName</i> option

In [ ]:
client = boto3.client('comprehend', region_name='us-east-1')
data_access_role_arn = "arn:aws:iam::XXXXXXXXX:role/service-role/AmazonSageMaker-ExecutionRole-20230418T155781"

input_csv_url = "s3://pro-ads-datalake/ml-training-data-csv/train_model.csv"
input_data_config = {"S3Uri": input_csv_url}
output_s3_url = "s3://pro-ads-datalake/ml-comp-predictions/"
output_data_config = {"S3Uri": output_s3_url}

# Create a document classifier
create_response = client.create_document_classifier(
    InputDataConfig=input_data_config, 
    OutputDataConfig=output_data_config,
    DataAccessRoleArn=data_access_role_arn, 
    DocumentClassifierName='TopicClassifierSept7', 
    LanguageCode='en'
)
print("Create response: %s\n", create_response)

# Check the status of the classifier
describe_response = client.describe_document_classifier(
    DocumentClassifierArn=create_response['DocumentClassifierArn'])
print("Describe response: %s\n", describe_response)

# List all classifiers in account 
list_response = client.list_document_classifiers()
print("List response: %s\n", list_response)

####  <font color="blue"> Check the status of the model creation job. Please provide the name of the model that is being built

In [ ]:
job_name = 'TopicClassifierSept7'

list_response = client.list_document_classifiers(
    Filter={'Status': 'TRAINING'})
    
print("\nJobs in TRAINING Status:\n", list_response)

list_response = client.list_document_classifiers(
    Filter={'DocumentClassifierName': 'TopicClassifierSept7'})
    
print("\nStatus of Job:", job_name, "\n", list_response)

#### <font color="blue"> Create endpoint.  Make sure to update the model-name in <i><font color="darkred"> ModelArn</i></font> <font color="blue">argument

In [ ]:
# import boto3
import time

client = boto3.client('comprehend', region_name='us-east-1')
data_access_role_arn = "arn:aws:iam::XXXXXXXXX:role/service-role/AmazonSageMaker-ExecutionRole-20230418T155781"

response = client.create_endpoint(
    EndpointName="rdsEmailClassifier",
    ModelArn="arn:aws:comprehend:us-east-1:XXXXXXXXX:document-classifier/TopicClassifierSept7",
    DesiredInferenceUnits=1,
    DataAccessRoleArn=data_access_role_arn
)
print(response)
'''
DocumentClassifierArn=create_response['DocumentClassifierArn']
realtime_endpoint_name = "TopicClassifierSept7"
'''

endpoint_arn = response['EndpointArn']
describe_response = client.describe_endpoint(EndpointArn=endpoint_arn)
while describe_response['EndpointProperties']['Status'] == 'CREATING':
    describe_response = client.describe_endpoint(EndpointArn=endpoint_arn)
    print(describe_response['EndpointProperties']['Status'])
    time.sleep(30)
describe_response

#### <font color="blue"> Test the newly built supervised model above (<i>TopicClassifierSept7</i>). Make sure the endpoint was created before testing.

In [ ]:
# import boto3

# client = boto3.client('comprehend', region_name='us-east-1')

cleanedTxt = "response file plan year rds support please review this issue again submitted an upload on received response file back today and all zeros are still appearing on response file continues to be missing reason codes and eligibility time frames please let me know why this error is still occurring as we need to get this solved to complete reopening by deadline thank you in advance for your assistance"
cleanedTxt = "unable to access site am unable to access site to complete my reconciliation it appears as though site is down"
cleanedTxt = "help thank you for your inquiry we're very sorry you're experiencing difficulties registering as account manager to begin your registration click account manager registration page or copy and paste following link into address bar of your browser refer to account manager registration for guidance on this process including step by step instructions"
cleanedTxtx = "change vendor for cost reporting thank you for your inquiry application id is currently in an incomplete status plan sponsors can only complete payment setup when an application is in any of following statuses approved reconciliation initiated reconciliation cost reporting opened reconciliation cost reporting closed reconciliation request completed as such you cannot complete payment setup for this application at this time after this application has been submitted and approved then you can complete payment setup"
cleanedTxt = "withdraw application rds application thank you for this information will update application status for application id to withdrawn an email notification will be sent once this action is complete"
cleanedTxt = "reconciliation issue with app id evansville please instruct me on why am unable to enter costs for this reconciliation manage final costs option is greyed out and not selectable even though finalize covered retirees step is completed"
cleanedtxt = "request we would like to receive so we can process our overpayment"
cleanedTxt = "re critical information about reconciliation deadline of your application in it had been determined there's no action needed by group health plan since there are no retirees for plan year we worked with from to close out cy application please refer to two emails for reference please let me know if you have any questions"
# cleanedTxt = "unable to request payment am unable to request payment at this time it doesn't allow me to confirm cost even after have reviewed it please advise me on how to proceed thank you school business executive cincinnatus csd opt opt"
# cleanedtxt = "plan id application cost reporter designee to whom can be of assistance our cost reporter for calendar year application is although he was previously assigned as designee st attached he stated he is unable to access application to report cost tried to remove his privilege and re enter him but website won't save and accept his designations to report costs and view send receive files attachment please review and advise as troubleshooting steps within guide has been unsuccessful"
# cleanedTxt = "upport request ref reporting interim costs cost reporting wanted to make sure taht vendor id is now properly identified on our application so that costs can be uploaded apparently costs had previoulsy been rejected believe that have fixed issue but wanted to confirm before next upload thank you for any assistance you can provide"
cleanedTxt = "rds reopening request approved this notice is to inform that retiree drug subsidy rds center has approved following reopening appeal tracking number determination that is subject of reopening request reconciliation final payment date reopening request was received by person who requested reopening new reconciliation deadline will allow plan sponsor to submit cost data beyond month timeframe set forth in and reopen pursuant to if plan sponsor wishes to increase amount of its subsidy for this application as stated in its reopening request plan sponsor must recomplete reconciliation by reconciliation deadline specified previously in this email in manner that reflects data and or subsidy amount plan sponsor specified in its request for reopening however plan sponsor is not required to recomplete reconciliation by that deadline if plan sponsor fails to or chooses not to will reinstate previous payment determination and amount of subsidy for application will not change if plan sponsor wishes to recomplete reconciliation read following for important information and steps plan sponsor must take for this application completing reconciliation changed status of this application to reconciliation initiated reconciliation step request list of covered retirees is marked with blue arrow to indicate that it is next step to be completed you are required to submit retiree list as described in to prior to requesting covered retiree list in reconciliation step this retiree list should contain at least one row for each beneficiary for which you are requesting subsidy please complete reconciliation step through step by reconciliation deadline specified in this email for more information go to reconciliation covered retirees after your retiree list has been processed and you have received retiree response file pursuant to matches names and identifying information for individuals submitted as qualifying covered retirees with database to determine which retirees are part eligible individuals who are not enrolled in part plan provides information concerning results of search such as names and other identifying information if necessary to sponsor or to designee please request covered retiree list in reconciliation step covered retiree list should be available within approximately minutes from time of request please download and evaluate covered retirees contained in this list if retiree record that was expected to be on covered retiree list crl is absent or if any of subsidy periods do not match plan sponsor's internal records discrepancy must be resolved before proceeding with remaining steps of reconciliation for more information on resolving discrepancies in covered retiree list go to request covered retiree lists the plan sponsor is required to review and agree to covered retiree list in reconciliation step consistent with sponsor will receive subsidy payment for each qualifying covered retiree enrolled with sponsor of qualified retiree prescription drug plan in plan year plan sponsors may only submit cost data for qualifying covered retirees benefit options and subsidy periods listed in covered retiree list that was downloaded from cost reporting cost data will be duplicated if same report is sent by both mainframe and data entry methods if this occurs one of reports must be deleted zeroed out to resolve duplication before reconciliation is submitted to after all corrected cost reports are received consistent with plan sponsor must close cost reporting and complete remaining steps of reconciliation by reconciliation deadline mainframe cost reports if revised mainframe cost report is submitted by same source that previously submitted cost report then revised cost report is replaced by old cost report if there are mainframe cost reports that plan sponsor wishes to retain plan sponsor does not have to do anything status remains mainframe submitted for more information go to reconciliation data entry cost reports the status of data entry cost reports has changed to data entry update entry required all data entry cost reports must be re saved or re submitted consistent with an account manager or designee may perform this action if they have appropriate privileges account managers with report costs privileges assigned data entry reporting method may access final cost data by selecting either enter update costs action within reconciliation step manage submission of final cost reports or final costs action on application list page designees with report costs privileges assigned data entry reporting method may access cost reports by selecting final costs on application list page for more information go to submit final cost data if further action is required on behalf of plan sponsor to effectuate this decision plan sponsor will be notified by if you need more information contact"

response = client.classify_document(
    Text=cleanedTxt, 
    EndpointArn='arn:aws:comprehend:us-east-1:029309145300:document-classifier-endpoint/rdsEmailClassifier'
)
print(response)